In [1]:
import auto_ml
from sklearn_pandas import DataFrameMapper, cross_val_score

import sklearn
import pandas as pd

In [2]:
def load_clean_transform(filename):
    data        = pd.read_csv(filename)
    file        = data.drop(["Name", "Ticket", "Cabin"], axis=1)
    file_age    = file["Age"].fillna(file["Age"].mean())
    file["Age"] = file_age
    
    return file

In [3]:
train = load_clean_transform("train.csv")
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [4]:
test = load_clean_transform("test.csv")
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S


In [5]:
train["Embarked"] = train["Embarked"].fillna("S")
# data = no_name_train.groupby("Embarked")
# data.count() # `S` is in majority
train.count()

PassengerId    891
Survived       891
Pclass         891
Sex            891
Age            891
SibSp          891
Parch          891
Fare           891
Embarked       891
dtype: int64

In [6]:
test["Fare"] = test["Fare"].fillna(test["Fare"].mean())
test.count()

PassengerId    418
Pclass         418
Sex            418
Age            418
SibSp          418
Parch          418
Fare           418
Embarked       418
dtype: int64

In [7]:
transformer = DataFrameMapper([
    ("PassengerId", None),
    ("Sex", sklearn.preprocessing.LabelBinarizer()),
    ("Embarked", sklearn.preprocessing.LabelBinarizer()),
], df_out=True)

In [8]:
only_numeric_train = transformer.fit_transform(train.copy())
only_numeric_train.head()

,PassengerId,Sex,Embarked_C,Embarked_Q,Embarked_S
0,1,1,0,0,1
1,2,0,1,0,0
2,3,0,0,0,1
3,4,0,0,0,1
4,5,1,0,0,1


In [9]:
only_numeric_test = transformer.fit_transform(test)
only_numeric_test.head()

,PassengerId,Sex,Embarked_C,Embarked_Q,Embarked_S
0,892,1,0,1,0
1,893,0,0,0,1
2,894,1,0,1,0
3,895,1,0,0,1
4,896,0,0,0,1


In [10]:
df_train = train.merge(only_numeric_train, on="PassengerId").drop(["Embarked", "Sex_x", "PassengerId"], axis=1)
df_train.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_y,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,1,0,0,1
1,1,1,38.0,1,0,71.2833,0,1,0,0
2,1,3,26.0,0,0,7.9250,0,0,0,1
3,1,1,35.0,1,0,53.1000,0,0,0,1
4,0,3,35.0,0,0,8.0500,1,0,0,1


In [11]:
df_test = test.merge(only_numeric_test, on="PassengerId").drop(["Embarked", "Sex_x", "PassengerId"], axis=1)
df_test.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_y,Embarked_C,Embarked_Q,Embarked_S
0,3,34.5,0,0,7.8292,1,0,1,0
1,3,47.0,1,0,7.0000,0,0,0,1
2,2,62.0,0,0,9.6875,1,0,1,0
3,3,27.0,0,0,8.6625,1,0,0,1
4,3,22.0,1,1,12.2875,0,0,0,1


In [12]:
column_descriptions = {
    "Survived": "output",
    "Sex": "categorical",
    "Pclass": "categorical"
}

model = auto_ml.Predictor("classifier", column_descriptions=column_descriptions)

In [19]:
# df_train, fl_data = sklearn.model_selection.train_test_split(df_train, test_size=0.33)

In [ ]:
model.train(df_train, optimize_final_model=True)

Welcome to auto_ml! We're about to go through and make sense of your data using machine learning, and give you a production-ready pipeline to get predictions with.

If you have any issues, or new feature ideas, let us know at http://auto.ml
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'warm_start': True, 'learning_rate': 0.1}
Running basic data cleaning
Fitting DataFrameVectorizer
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'warm_start': True, 'learning_rate': 0.1}


********************************************************************************************
Optimizing the hyperparameters for your model now
About to run EvolutionaryAlgorithmSearchCV to find the optimal hyperparameters for

[32] random_holdout_set_from_training_data's score is: -0.168
[29] random_holdout_set_from_training_data's score is: -0.173
[22] random_holdout_set_from_training_data's score is: -0.226
[20] random_holdout_set_from_training_data's score is: -0.107
[33] random_holdout_set_from_training_data's score is: -0.168
[30] random_holdout_set_from_training_data's score is: -0.171
[23] random_holdout_set_from_training_data's score is: -0.226
[21] random_holdout_set_from_training_data's score is: -0.109
[31] random_holdout_set_from_training_data's score is: -0.17
[34] random_holdout_set_from_training_data's score is: -0.168
[32] random_holdout_set_from_training_data's score is: -0.168
[24] random_holdout_set_from_training_data's score is: -0.226
[22] random_holdout_set_from_training_data's score is: -0.107
[35] random_holdout_set_from_training_data's score is: -0.166
[33] random_holdout_set_from_training_data's score is: -0.168
[25] random_holdout_set_from_training_data's score is: -0.226
[36] rand

[82] random_holdout_set_from_training_data's score is: -0.151
[74] random_holdout_set_from_training_data's score is: -0.152
[16] random_holdout_set_from_training_data's score is: -0.163
[84] random_holdout_set_from_training_data's score is: -0.151
[76] random_holdout_set_from_training_data's score is: -0.152
[50] random_holdout_set_from_training_data's score is: -0.221
[17] random_holdout_set_from_training_data's score is: -0.166
[78] random_holdout_set_from_training_data's score is: -0.152
[86] random_holdout_set_from_training_data's score is: -0.15
[52] random_holdout_set_from_training_data's score is: -0.221
[80] random_holdout_set_from_training_data's score is: -0.151
[18] random_holdout_set_from_training_data's score is: -0.168
[82] random_holdout_set_from_training_data's score is: -0.151
[54] random_holdout_set_from_training_data's score is: -0.22
[88] random_holdout_set_from_training_data's score is: -0.15
[84] random_holdout_set_from_training_data's score is: -0.151
[56] random

[98] random_holdout_set_from_training_data's score is: -0.212
[18] random_holdout_set_from_training_data's score is: -0.192
[166] random_holdout_set_from_training_data's score is: -0.14
[169] random_holdout_set_from_training_data's score is: -0.141
[172] random_holdout_set_from_training_data's score is: -0.141
[19] random_holdout_set_from_training_data's score is: -0.19
[100] random_holdout_set_from_training_data's score is: -0.212
[157] random_holdout_set_from_training_data's score is: -0.141
[103] random_holdout_set_from_training_data's score is: -0.212
[175] random_holdout_set_from_training_data's score is: -0.14
[20] random_holdout_set_from_training_data's score is: -0.186
[160] random_holdout_set_from_training_data's score is: -0.141
[106] random_holdout_set_from_training_data's score is: -0.211
[21] random_holdout_set_from_training_data's score is: -0.182
[163] random_holdout_set_from_training_data's score is: -0.141
[178] random_holdout_set_from_training_data's score is: -0.139


[238] random_holdout_set_from_training_data's score is: -0.133
[255] random_holdout_set_from_training_data's score is: -0.133
[98] random_holdout_set_from_training_data's score is: -0.126
[157] random_holdout_set_from_training_data's score is: -0.202
[260] random_holdout_set_from_training_data's score is: -0.133
[100] random_holdout_set_from_training_data's score is: -0.127
[265] random_holdout_set_from_training_data's score is: -0.134
[270] random_holdout_set_from_training_data's score is: -0.134
[103] random_holdout_set_from_training_data's score is: -0.126
[275] random_holdout_set_from_training_data's score is: -0.134
[241] random_holdout_set_from_training_data's score is: -0.133
[160] random_holdout_set_from_training_data's score is: -0.202
[280] random_holdout_set_from_training_data's score is: -0.134
[285] random_holdout_set_from_training_data's score is: -0.134
[163] random_holdout_set_from_training_data's score is: -0.201
[106] random_holdout_set_from_training_data's score is: 

...

[56] random_holdout_set_from_training_data's score is: -0.131
The number of estimators that were the best for this training dataset: 26
[58] random_holdout_set_from_training_data's score is: -0.13
[50] random_holdout_set_from_training_data's score is: -0.166
[60] random_holdout_set_from_training_data's score is: -0.13
The best score on the holdout set: -0.12233055503756345
[62] random_holdout_set_from_training_data's score is: -0.129
[115] random_holdout_set_from_training_data's score is: -0.161
[52] random_holdout_set_from_training_data's score is: -0.164
[64] random_holdout_set_from_training_data's score is: -0.127
[54] random_holdout_set_from_training_data's score is: -0.162
[118] random_holdout_set_from_training_data's score is: -0.16
[66] random_holdout_set_from_training_data's score is: -0.127
[CV]  model__max_features=sqrt, model__learning_rate=0.1, model__loss=exponential, model__max_depth=4, _scorer=<auto_ml.utils_scoring.ClassificationScorer object at 0x7f2a476d8cd0>, model__

[157] random_holdout_set_from_training_data's score is: -0.15
[12] random_holdout_set_from_training_data's score is: -0.161
[42] random_holdout_set_from_training_data's score is: -0.234
[13] random_holdout_set_from_training_data's score is: -0.16
[43] random_holdout_set_from_training_data's score is: -0.234
[92] random_holdout_set_from_training_data's score is: -0.137
[44] random_holdout_set_from_training_data's score is: -0.233
[14] random_holdout_set_from_training_data's score is: -0.162
[45] random_holdout_set_from_training_data's score is: -0.233
[160] random_holdout_set_from_training_data's score is: -0.15
[46] random_holdout_set_from_training_data's score is: -0.233
[15] random_holdout_set_from_training_data's score is: -0.163
[94] random_holdout_set_from_training_data's score is: -0.135
[16] random_holdout_set_from_training_data's score is: -0.161
[47] random_holdout_set_from_training_data's score is: -0.233
[48] random_holdout_set_from_training_data's score is: -0.233
[17] rand

[35] random_holdout_set_from_training_data's score is: -0.149
The number of estimators that were the best for this training dataset: 15
[148] random_holdout_set_from_training_data's score is: -0.119
The best score on the holdout set: -0.13331335584738435
[199] random_holdout_set_from_training_data's score is: -0.142
[109] random_holdout_set_from_training_data's score is: -0.227
[151] random_holdout_set_from_training_data's score is: -0.117
[202] random_holdout_set_from_training_data's score is: -0.142
[112] random_holdout_set_from_training_data's score is: -0.226
[CV]  model__max_features=None, model__learning_rate=0.1, model__loss=deviance, model__max_depth=7, _scorer=<auto_ml.utils_scoring.ClassificationScorer object at 0x7f2a805b4c50>, model__n_estimators=100, model__subsample=1.0, score=-0.15433648452, total=   0.7s
[115] random_holdout_set_from_training_data's score is: -0.226
[154] random_holdout_set_from_training_data's score is: -0.117
[205] random_holdout_set_from_training_dat

[280] random_holdout_set_from_training_data's score is: -0.108
[365] random_holdout_set_from_training_data's score is: -0.133
[285] random_holdout_set_from_training_data's score is: -0.108
[370] random_holdout_set_from_training_data's score is: -0.134
[241] random_holdout_set_from_training_data's score is: -0.215
[375] random_holdout_set_from_training_data's score is: -0.134
[290] random_holdout_set_from_training_data's score is: -0.108
[295] random_holdout_set_from_training_data's score is: -0.108
[244] random_holdout_set_from_training_data's score is: -0.215
[300] random_holdout_set_from_training_data's score is: -0.108
[305] random_holdout_set_from_training_data's score is: -0.108
[247] random_holdout_set_from_training_data's score is: -0.215
[380] random_holdout_set_from_training_data's score is: -0.133
[310] random_holdout_set_from_training_data's score is: -0.108
[250] random_holdout_set_from_training_data's score is: -0.215
[315] random_holdout_set_from_training_data's score is:

[620] random_holdout_set_from_training_data's score is: -0.189
[630] random_holdout_set_from_training_data's score is: -0.189
[640] random_holdout_set_from_training_data's score is: -0.188
[650] random_holdout_set_from_training_data's score is: -0.187
[660] random_holdout_set_from_training_data's score is: -0.187
[670] random_holdout_set_from_training_data's score is: -0.186
[680] random_holdout_set_from_training_data's score is: -0.186
[690] random_holdout_set_from_training_data's score is: -0.185
[700] random_holdout_set_from_training_data's score is: -0.185
[710] random_holdout_set_from_training_data's score is: -0.184
[720] random_holdout_set_from_training_data's score is: -0.184
[730] random_holdout_set_from_training_data's score is: -0.183
[740] random_holdout_set_from_training_data's score is: -0.183
[750] random_holdout_set_from_training_data's score is: -0.182
[760] random_holdout_set_from_training_data's score is: -0.182
[770] random_holdout_set_from_training_data's score is:

In [21]:
model.score(df_train, df_train["Survived"])

Here is our brier-score-loss, which is the default value we optimized for while training, and is the value returned from .score() unless you requested a custom scoring metric
It is a measure of how close the PROBABILITY predictions are.
0.0908

Here is the trained estimator's overall accuracy (when it predicts a label, how frequently is that the correct label?)
89.7%

Here is a confusion matrix showing predictions vs. actuals by label:
Predicted >    0    1  All
v Actual v                
0            529   20  549
1             72  270  342
All          601  290  891

Here is predictive value by class:
('Class: ', 0, '=', 0.8801996672212978)
('Class: ', 1, '=', 0.9310344827586207)
+----------------+-----------------------------------+--------------------------------+
| Bucket Edges   |   Predicted Probability Of Bucket |   Actual Probability of Bucket |
|----------------+-----------------------------------+--------------------------------|
| (0.051, 0.102] |                           

-0.090814334599907648

In [22]:
submission = pd.DataFrame({"PassengerId": [x for x in range(892, 892 + 418)], "Survived": model.predict(df_test)})

with open("submission_19.csv", "w") as f_ptr:
    f_ptr.write(submission.to_csv(index=False))